# 5. Image enhancement – histogram equalization

In [1]:
import cv2
import numpy as np

IMAGE_PATH = "images/lowContrast_01.jpg"
IMAGE_PATH_COLOR = "images/lowContrast_02.jpg"

image = cv2.imread(IMAGE_PATH, cv2.IMREAD_GRAYSCALE)
image_color = cv2.imread(IMAGE_PATH_COLOR, cv2.IMREAD_COLOR)

HIST_SIZE = 256
HIST_RANGE = (0, 256)
ACCUMULATE = False
HIST_W = 512
HIST_H = 400
BIN_W = int(round(HIST_W / HIST_SIZE))

**a)** Take a low contrast grayscale image and plot its histogram.

In [2]:
def histogram(image):
    hist = cv2.calcHist(
        image, [0], None, [HIST_SIZE], HIST_RANGE, accumulate=ACCUMULATE
    )
    hist_image = np.zeros((HIST_H, HIST_W, 3), np.uint8)
    cv2.normalize(hist, hist, 0, HIST_H, cv2.NORM_MINMAX)

    for i in range(1, HIST_SIZE):
        cv2.line(
            hist_image,
            (BIN_W * (i - 1), HIST_H - int(hist[i - 1])),
            (BIN_W * (i), HIST_H - int(hist[i])),
            (255, 255, 255),
            thickness=2,
        )
    return hist_image


hist_image = histogram(image)
cv2.imshow("Source image", image)
cv2.imshow("Histogram", hist_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

**b)** Enhance the image constrast and show the resulting enhanced images and their histograms using:
- **b1)** simple histogram equalization

In [3]:
equalized = cv2.equalizeHist(image)
hist_equalized = histogram(equalized)
hist_source = histogram(image)

cv2.imshow("Source", image)
cv2.imshow("Source Histogram", hist_source)
cv2.imshow("Result", equalized)
cv2.imshow("Result Histogram", hist_equalized)
cv2.waitKey(0)
cv2.destroyAllWindows()

- **b2)** CLAHE

In [4]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
cl1 = clahe.apply(image)
hist_clahe = histogram(cl1)

cv2.imshow("Source", image)
cv2.imshow("Source Histogram", hist_source)
cv2.imshow("CLAHE", cl1)
cv2.imshow("CLAHE Histogram", hist_clahe)
cv2.waitKey(0)
cv2.destroyAllWindows()

**c)** Repeat the previous operations on a color image.

In [7]:
def color_histogram(image):
    b_hist = cv2.calcHist(
        [image], [0], None, [HIST_SIZE], HIST_RANGE, accumulate=ACCUMULATE
    )
    g_hist = cv2.calcHist(
        [image], [1], None, [HIST_SIZE], HIST_RANGE, accumulate=ACCUMULATE
    )
    r_hist = cv2.calcHist(
        [image], [2], None, [HIST_SIZE], HIST_RANGE, accumulate=ACCUMULATE
    )

    hist_image = np.zeros((HIST_H, HIST_W, 3), np.uint8)

    cv2.normalize(b_hist, b_hist, 0, HIST_H, cv2.NORM_MINMAX)
    cv2.normalize(g_hist, g_hist, 0, HIST_H, cv2.NORM_MINMAX)
    cv2.normalize(r_hist, r_hist, 0, HIST_H, cv2.NORM_MINMAX)

    for i in range(1, HIST_SIZE):
        cv2.line(
            hist_image,
            (BIN_W * (i - 1), HIST_H - int(b_hist[i - 1])),
            (BIN_W * (i), HIST_H - int(b_hist[i])),
            (255, 0, 0),
            thickness=2,
        )
        cv2.line(
            hist_image,
            (BIN_W * (i - 1), HIST_H - int(g_hist[i - 1])),
            (BIN_W * (i), HIST_H - int(g_hist[i])),
            (0, 255, 0),
            thickness=2,
        )
        cv2.line(
            hist_image,
            (BIN_W * (i - 1), HIST_H - int(r_hist[i - 1])),
            (BIN_W * (i), HIST_H - int(r_hist[i])),
            (0, 0, 255),
            thickness=2,
        )
    return hist_image


hsv_image = cv2.cvtColor(image_color, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv_image)
v_equalized = cv2.equalizeHist(v)
equalized_hsv = cv2.merge([h, s, v_equalized])
equalized_image = cv2.cvtColor(equalized_hsv, cv2.COLOR_HSV2BGR)

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
v_clahe = clahe.apply(v)
clahe_hsv = cv2.merge([h, s, v_clahe])
clahe_image = cv2.cvtColor(clahe_hsv, cv2.COLOR_HSV2BGR)


hist_image = color_histogram(image_color)
cv2.imshow("Source image", image_color)
cv2.imshow("Source Histogram", hist_image)

hist_equalized = color_histogram(equalized_hsv)
cv2.imshow("Equalized image", equalized_image)
cv2.imshow("Equalized Histogram", hist_equalized)

hist_clahe = color_histogram(clahe_hsv)
cv2.imshow("CLAHE image", clahe_image)
cv2.imshow("CLAHE Histogram", hist_clahe)

cv2.waitKey(0)
cv2.destroyAllWindows()